#### In this step, we want to transform the time series data into Time-series data

In [2]:
import pandas as pd
import numpy as np

In [3]:
rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')
rides.head(5)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [4]:
## Then we'll add another column to the dataframe, whic represents the rounded hour for each hour

In [5]:
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

/tmp/ipykernel_30875/2261653417.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')


,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [6]:
### Now we want to group, per pickup hour, and per location id. So that we know the number of rides per location id per pickup hour

In [9]:
agg_rides = rides.groupby(['pickup_hour','pickup_location_id']).size().reset_index()